# Visualize Point Clouds

The scene coordinates aka point clouds can be visualized in this notebook.
We had already extracted the `process/measurement/filtered` scene coordinates $\theta_t$ ,  for `fire` and `heads` data in the `7 scenes dataset`

In [2]:
from glob import glob
import numpy as np
import open3d
import time
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Note:  You can skip Step 1 and 2 if you do not want to run the evaluation. You can use our `output` folder from [here](https://drive.google.com/drive/u/1/folders/1cMDvOlGF1XWl98dUlTmkPmoREFFFfQ2D) which contains the estimated scene coordinates.


# Step 1: Generate the IMAGE and LABEL list, in the ```input/``` folder


### [UNCOMMENT BELOW CODE]: TO GENERATE LABEL LIST and IMAGE LIST

- make sure the groundtruth labels are downloaded from the links in [main repository](https://github.com/zlthinker/KFNet) and stored in ```inputs/``` folder as ```$folder$-label ```

- make sure the images are downloaded from the 7scenes dataset [website](https://www.microsoft.com/en-us/research/project/rgb-d-dataset-7-scenes/) and saved under ``` inputs/``` as ``` $folder$-input-images```

- The `label_list.txt` and the `image_list.txt` need to be saved in the ``` inputs/``` directory. Also copy the `transform.txt` from ```$folder$-label ``` and paste in ``` inputs/```

In [3]:

# folder = 'fire'
# sequence = 'seq-01'

# [UNCOMMENT BELOW CODE]: TO GENERATE LABEL LIST and IMAGE LIST

# BIN_DIR = f"./input/{folder}-label/{sequence}/bin"
# N = len(glob(f"{BIN_DIR}/*.bin"))
# files = [f"{BIN_DIR}/{str(i)}.bin"  for i in range(N)]
# np.savetxt('./input/label_list.txt', np.array(files), fmt='%s')


# N = len(glob(f"./input/{folder}-input-images/{sequence}/*.color.png"))
# files = sorted(glob(f"./input/{folder}-input-images/{sequence}/*.color.png"))
# np.savetxt(f'./input/image_list.txt', np.array(files), fmt='%s')

# Step2: Run Evaluation
- download the models and place in the models folder.
- download the data  sequence from 7 scenes for images and the prepared groundtruth labels from the authors. Further details are given [here](https://github.com/zlthinker/KFNet)
- use docker in the terminal to load to the workspace and execute evaluation.
- run the evaluation script using the following command

- Example:
 `python KFNet/eval.py --input_folder ./input --output_folder ./output/fire --model_folder ./models/KFNet/heads --scene fire`
- note the distance errors
- save the scene coords in the output folder


# Visualize Point Clouds
- specify the scene and the model  

## Functions

In [4]:
def pointcloud_to_depth_map(pointcloud: np.ndarray, theta_res=150, phi_res=32, max_depth=50, phi_min_degrees=60,
                            phi_max_degrees=100) -> np.ndarray:
    """
        All params are set so they match default carla lidar settings
    """
    assert pointcloud.shape[1] == 3, 'Must have (N, 3) shape'
    assert len(pointcloud.shape) == 2, 'Must have (N, 3) shape'

    xs = pointcloud[:, 0]
    ys = pointcloud[:, 1]
    zs = pointcloud[:, 2]

    rs = np.sqrt(np.square(xs) + np.square(ys) + np.square(zs))

    phi_min = np.deg2rad(phi_min_degrees)
    phi_max = np.deg2rad(phi_max_degrees)
    phi_range = phi_max - phi_min
    phis = np.arccos(zs / rs)

    THETA_MIN = -np.pi
    THETA_MAX = np.pi
    THETA_RANGE = THETA_MAX - THETA_MIN
    thetas = np.arctan2(xs, ys)

    phi_indices = ((phis - phi_min) / phi_range) * (phi_res - 1)
    phi_indices = np.rint(phi_indices).astype(np.int16)

    theta_indices = ((thetas - THETA_MIN) / THETA_RANGE) * theta_res
    theta_indices = np.rint(theta_indices).astype(np.int16)
    theta_indices[theta_indices == theta_res] = 0

    normalized_r = rs / max_depth

    canvas = np.ones(shape=(theta_res, phi_res), dtype=np.float32)
    # We might need to filter out out-of-bound phi values, if min-max degrees doesnt match lidar settings
    canvas[theta_indices, phi_indices] = normalized_r

    depth_map = canvas * 256
    depth_map = np.flip(depth_map, axis=1) # so floor is down
    depth_map = np.swapaxes(depth_map, 0, 1)    
    return depth_map

import math
def rotation_matrix(angle, direction, point=None):
    sina = math.sin(angle)
    cosa = math.cos(angle)
    # rotation matrix around unit vector
    R = np.diag([cosa, cosa, cosa])
    R += np.outer(direction, direction) * (1.0 - cosa)
    print( direction, direction.dtype)
    direction *= sina
    R += np.array([[ 0.0,         -direction[2],  direction[1]],
                      [ direction[2], 0.0,          -direction[0]],
                      [-direction[1], direction[0],  0.0]])
    M = np.identity(4)
    M[:3, :3] = R
    if point is not None:
        # rotation not around origin
        point = np.array(point[:3], dtype=np.float64, copy=False)
        M[:3, 3] = point - np.dot(R, point)
    return M

def filter_pointclouds(npy_file, thres=None):
    # added auto-thresholding
    map = np.load(npy_file)
    coords = map[:, :, 0:3]
    confidences = map[:, :, 3]
    if not thres:
        thres = 0.3*np.max(confidences) 

    coords = np.reshape(coords, (-1, 3))
    confidences = confidences.flatten().tolist()

    coords_filtered = []
    for i in range(len(confidences)):
        if confidences[i] > thres:
            coords_filtered.append(coords[i])
    coords_filtered = np.vstack(coords_filtered)
    return coords_filtered

def read_pointclouds(npy_file, thres):
    coords_filtered = filter_pointclouds(npy_file, thres)
    print ('Load #points:', coords_filtered.shape[0], ' from', npy_file)
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(coords_filtered)
    return pcd

## To visualize the states.

In [6]:
folder = 'fire' # heads
type =  'measured' # filtered # process # measured

# To visualize process states
files = sorted(glob(f"output/{folder}/{type}/*.npy"))
coordfiles = [f"output/{folder}/{type}/coord_{str(i)}.npy" for i in range(len(files))]

print(f"Number of point clouds in {type}: ", len(coordfiles))
print("shape of point clouds:", np.load(coordfiles[0]).shape)

Number of point clouds in measured:  1000
shape of point clouds: (60, 80, 4)


## Run Visualizer

In [7]:
rotate= 180 #deg
rotation = rotation_matrix(math.pi * rotate / 180.0, np.array([1.0, 0.0, 0.0]))
print('rotation', rotation)


vis = open3d.visualization.Visualizer()
vis.create_window()

line_set = open3d.geometry.LineSet()
line_set.transform(rotation)
for i in range(0, len(coordfiles), 1 ):
    npy_file = coordfiles[i]
    pcd = read_pointclouds(npy_file, None)
    print()
    pcd.transform(rotation)

    vis.add_geometry(pcd)
    vis.update_geometry(None)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.05 * 1)
vis.destroy_window()


[1. 0. 0.] float64
rotation [[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00 -1.0000000e+00 -1.2246468e-16  0.0000000e+00]
 [ 0.0000000e+00  1.2246468e-16 -1.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
Load #points: 3361  from output/fire/measured/coord_0.npy

Load #points: 3356  from output/fire/measured/coord_1.npy

Load #points: 3461  from output/fire/measured/coord_2.npy

Load #points: 3459  from output/fire/measured/coord_3.npy

Load #points: 3565  from output/fire/measured/coord_4.npy

Load #points: 3573  from output/fire/measured/coord_5.npy

Load #points: 3642  from output/fire/measured/coord_6.npy

Load #points: 3527  from output/fire/measured/coord_7.npy

Load #points: 3458  from output/fire/measured/coord_8.npy

Load #points: 3511  from output/fire/measured/coord_9.npy

Load #points: 3531  from output/fire/measured/coord_10.npy

Load #points: 3451  from output/fire/measured/coord_11.npy

Load #poi

KeyboardInterrupt: 